In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
import tensorflow as tf
import keras
import seaborn as sns 
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

2023-04-13 20:51:25.260569: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('Resources_Clean/Result_Protein_Peptide_Combine.csv')

In [3]:
df.head()

,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,On,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,...,62329,62437,62723,62732,62792,63875,63889,64669,64674,65043
0,0,10.0,6.0,15.0,0.0,0,8984260.0,53855.6,8579740.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,6,8.0,10.0,34.0,0.0,0,8279770.0,45251.9,8655890.0,49927.5,...,0,0,0,0,0,0,0,0,0,0
2,12,10.0,10.0,41.0,0.0,1,8382390.0,53000.9,8995640.0,45519.2,...,0,0,0,0,0,0,0,0,0,0
3,36,17.0,18.0,51.0,0.0,1,10671500.0,58108.4,9985420.0,52374.0,...,0,0,0,0,0,0,0,0,0,0
4,6,8.0,2.0,21.0,0.0,0,6177730.0,42682.6,3596660.0,25698.8,...,0,0,0,0,0,0,0,0,0,0


### Split & Train

In [4]:
# Getting X, Y columns 
y_col = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']
x_col = [x for x in df.columns if x not in y_col]

# Splitting into X and Y 

X = df[x_col]
Y = df[y_col]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

### Scaling Data 

In [5]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
# Conversion to dataframe

scaled_test_df = pd.DataFrame(X_test_scaled, columns = X_test.columns)
scaled_train_df = pd.DataFrame(X_train_scaled, columns = X_train.columns)

### Loading Features 

In [7]:
import ast

In [8]:
df_col = pd.read_csv('Resources_Clean/features.csv')

In [9]:
df_col.head()

,features
0,"['On', 'FIYGGC(UniMod_4)GGNR', 'P43121', 'TLKI..."
1,"['HLSLLTTLSNR', 'VRGGEGTGYFVDFSVR', 'Q14515', ..."
2,"['On', 'FIYGGC(UniMod_4)GGNR', 'P43121', 'TLKI..."
3,"['HLSLLTTLSNR', 'VKDLATVYVDVLK', 'VRGGEGTGYFVD..."
4,"['On', 'FIYGGC(UniMod_4)GGNR', 'P43121', 'TLKI..."


In [10]:
df_col['features'] = df_col['features'].apply(ast.literal_eval)

In [11]:
remove_corr = df_col['features'].tolist()
remove_corr[0]

['On',
 'FIYGGC(UniMod_4)GGNR',
 'P43121',
 'TLKIENVSYQDKGNYR',
 'P17174',
 'Q06481',
 'O15240',
 'QQETAAAETETR',
 'P05060',
 '40650',
 'P13521',
 'ALEYIENLR',
 'AYQGVAAPFPK',
 'P40925',
 'FVEGLPINDFSR',
 'O00533',
 'VNGSPVDNHPFAGDVVFPR',
 'EAEEETTNDNGVLVLEPARK',
 'VRQGQGQSEPGEYEQR',
 'IGADFLAR',
 'LVFFAEDVGSNK',
 'Q9BY67',
 'VHKEDDGVPVIC(UniMod_4)QVEHPAVTGNLQTQR',
 'FFLC(UniMod_4)QVAGDAK',
 'GEAGAPGEEDIQGPTK',
 'KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK',
 'VFNTPEGVPSAPSSLK',
 'RLEGQEEEEDNRDSSMK',
 'Q6UXD5',
 'EGDMLTLFDGDGPSAR',
 'GATLALTQVTPQDER',
 'QHVVYGPWNLPQSSYSHLTR',
 'LQAEAFQAR',
 'LEEQAQQIR',
 'THLGEALAPLSK',
 'LDEVKEQVAEVR',
 'KLSENTDFLAPGVSSFTDSNQQESITK',
 'P04180',
 'IEIPSSVQQVPTIIK',
 '13618',
 'P10645',
 'P02787',
 'P14618',
 'IKPVFIEDANFGR',
 'SEALAVDGAGKPGAEEAQDPEGK',
 'Q8NBJ4',
 'DTINLLDQR',
 'LIADLGSTSITNLGFR',
 'ISYGNDALMPSLTETK',
 'VGGVQSLGGTGALR',
 'C(UniMod_4)APFFYGGC(UniMod_4)GGNR',
 'P05067',
 'LLRDPADASEAHESSSR',
 'LDIDSPPITAR',
 'STNLHDYGMLLPC(UniMod_4)GIDK',
 'Q

In [12]:
for i in remove_corr:
    print(len(i))

382
1240
754
1181
1080
1102


### Function to input Features

In [13]:
from keras.layers import LeakyReLU

def nn_test(range_):

  # For storing results 
  results = {}

  for index in range_:
  # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
    nn_model = tf.keras.models.Sequential()
    nn_model.add(tf.keras.layers.Dense(units=36, activation = LeakyReLU(alpha= 0.1), input_dim = len(scaled_train_df[remove_corr[index]].columns)))

    # First hidden layer
    nn_model.add(tf.keras.layers.Dense(units=128, activation="relu"))
    # Second hidden layer
    nn_model.add(tf.keras.layers.Dense(units=128, activation="relu"))
    # Third hidden layer
    nn_model.add(tf.keras.layers.Dense(units=128, activation="relu"))

    # Output layer
    nn_model.add(tf.keras.layers.Dense(units= 4, activation="relu"))

    #Compile Model
    nn_model.compile(loss="mean_absolute_error", optimizer="adam", metrics= ['mean_squared_error'])
    
    # Stop Early
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience= 20)
    
    # Train Model
    nn_model.fit(scaled_train_df[remove_corr[index]], y_train, epochs = 200, callbacks = [stop_early])
    
    #load model
    #nn_model = keras.models.load_model(f'FeatureSelection/feature_select_nn_{index}')

    # Evaluate Model
    model_loss, model_accuracy = nn_model.evaluate(scaled_test_df[remove_corr[index]] , y_test, verbose=2) 

    nn_model.save(f'FeatureSelection/feature_select_nn_{index}')

    results[index] = {'model_loss': model_loss, 'model_accuracy': model_accuracy}
    
    print(results[index])
 
  return results 

In [14]:
range_ = [i for i in range(0, 6)]

In [15]:
results = nn_test(range_)

Epoch 1/200
27/27 [==============================] - 1s 2ms/step - loss: 6.9682 - mean_squared_error: 152.0551
Epoch 2/200
27/27 [==============================] - 0s 2ms/step - loss: 6.6600 - mean_squared_error: 145.5045
Epoch 3/200
27/27 [==============================] - 0s 2ms/step - loss: 6.3220 - mean_squared_error: 139.1542
Epoch 4/200
27/27 [==============================] - 0s 1ms/step - loss: 6.1268 - mean_squared_error: 136.8044
Epoch 5/200
27/27 [==============================] - 0s 1ms/step - loss: 5.9940 - mean_squared_error: 135.4007
Epoch 6/200
27/27 [==============================] - 0s 2ms/step - loss: 5.8900 - mean_squared_error: 134.5303
Epoch 7/200
27/27 [==============================] - 0s 2ms/step - loss: 5.7938 - mean_squared_error: 133.6913
Epoch 8/200
27/27 [==============================] - 0s 1ms/step - loss: 5.6960 - mean_squared_error: 132.9544
Epoch 9/200
27/27 [==============================] - 0s 1ms/step - loss: 5.6272 - mean_squared_error: 132.4086
E

27/27 [==============================] - 0s 2ms/step - loss: 4.6681 - mean_squared_error: 126.1555
Epoch 75/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6821 - mean_squared_error: 126.1939
Epoch 76/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6645 - mean_squared_error: 126.1527
Epoch 77/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6604 - mean_squared_error: 126.1293
Epoch 78/200
27/27 [==============================] - 0s 3ms/step - loss: 4.6705 - mean_squared_error: 126.1552
Epoch 79/200
27/27 [==============================] - 0s 3ms/step - loss: 4.6414 - mean_squared_error: 126.0939
Epoch 80/200
27/27 [==============================] - 0s 4ms/step - loss: 4.6577 - mean_squared_error: 126.1415
Epoch 81/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6451 - mean_squared_error: 126.1045
Epoch 82/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6561 - mean_squared_error: 126.1268
Epoch

27/27 [==============================] - 0s 2ms/step - loss: 4.6139 - mean_squared_error: 126.0595
Epoch 147/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6019 - mean_squared_error: 126.0405
Epoch 148/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6053 - mean_squared_error: 126.0411
Epoch 149/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6060 - mean_squared_error: 126.0525
Epoch 150/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6313 - mean_squared_error: 126.0986
Epoch 151/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6265 - mean_squared_error: 126.0818
Epoch 152/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6400 - mean_squared_error: 126.1083
Epoch 153/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6268 - mean_squared_error: 126.0792
Epoch 154/200
27/27 [==============================] - 0s 2ms/step - loss: 4.6153 - mean_squared_error: 126.06

INFO:tensorflow:Assets written to: FeatureSelection/feature_select_nn_0/assets


INFO:tensorflow:Assets written to: FeatureSelection/feature_select_nn_0/assets


...

27/27 [==============================] - 0s 2ms/step - loss: 0.8069 - mean_squared_error: 2.8363
Epoch 75/200
27/27 [==============================] - 0s 2ms/step - loss: 0.7703 - mean_squared_error: 2.7200
Epoch 76/200
27/27 [==============================] - 0s 2ms/step - loss: 0.7442 - mean_squared_error: 2.6597
Epoch 77/200
27/27 [==============================] - 0s 2ms/step - loss: 0.7914 - mean_squared_error: 2.8084
Epoch 78/200
27/27 [==============================] - 0s 2ms/step - loss: 0.7756 - mean_squared_error: 2.7550
Epoch 79/200
27/27 [==============================] - 0s 2ms/step - loss: 0.7372 - mean_squared_error: 2.6460
Epoch 80/200
27/27 [==============================] - 0s 2ms/step - loss: 0.7465 - mean_squared_error: 2.6536
Epoch 81/200
27/27 [==============================] - 0s 2ms/step - loss: 0.7635 - mean_squared_error: 2.7261
Epoch 82/200
27/27 [==============================] - 0s 2ms/step - loss: 0.8002 - mean_squared_error: 2.8134
Epoch 83/200
27/27 [===

27/27 [==============================] - 0s 2ms/step - loss: 0.7824 - mean_squared_error: 2.7346
Epoch 149/200
27/27 [==============================] - 0s 2ms/step - loss: 0.7009 - mean_squared_error: 2.5363
Epoch 150/200
27/27 [==============================] - 0s 2ms/step - loss: 0.6816 - mean_squared_error: 2.4400
Epoch 151/200
27/27 [==============================] - 0s 2ms/step - loss: 0.7354 - mean_squared_error: 2.6441
Epoch 152/200
27/27 [==============================] - 0s 2ms/step - loss: 0.6813 - mean_squared_error: 2.4578
Epoch 153/200
27/27 [==============================] - 0s 2ms/step - loss: 0.6758 - mean_squared_error: 2.4753
Epoch 154/200
27/27 [==============================] - 0s 2ms/step - loss: 0.6573 - mean_squared_error: 2.4317
Epoch 155/200
27/27 [==============================] - 0s 2ms/step - loss: 0.6371 - mean_squared_error: 2.3727
Epoch 156/200
27/27 [==============================] - 0s 2ms/step - loss: 0.6509 - mean_squared_error: 2.3830
Epoch 157/200
2

INFO:tensorflow:Assets written to: FeatureSelection/feature_select_nn_5/assets


INFO:tensorflow:Assets written to: FeatureSelection/feature_select_nn_5/assets


{'model_loss': 3.9384572505950928, 'model_accuracy': 44.26483154296875}


In [16]:
results

{0: {'model_loss': 6.512526035308838, 'model_accuracy': 163.75912475585938},
 1: {'model_loss': 3.7468717098236084, 'model_accuracy': 40.092899322509766},
 2: {'model_loss': 3.4978766441345215, 'model_accuracy': 36.208038330078125},
 3: {'model_loss': 4.014716148376465, 'model_accuracy': 42.866214752197266},
 4: {'model_loss': 3.3606693744659424, 'model_accuracy': 32.200077056884766},
 5: {'model_loss': 3.9384572505950928, 'model_accuracy': 44.26483154296875}}

### Optimization

In [17]:
best_model =  keras.models.load_model('FeatureSelection/feature_select_nn_4')

In [ ]:
 # New Sequential model with hyperparameter options
def create_model(hp):
    
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu', 'tanh', 'swish', 'gelu', 'sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=256,
        step=7), activation= activation, input_dim= len(best_features)))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=32,
            max_value=256,
            step=16),
            activation=activation))
    
    # optimizer 
    Optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1.0, 0.1, 0.01, 0.001, 0.0001]))
    
    nn_model.add(tf.keras.layers.Dense(units=4, activation= hp.Choice('activation', ['relu'])))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer = Optimizer, metrics=["accuracy"])
    
    return nn_model